# Cleaning data

In [ ]:
#!/usr/bin/env python3

import pandas as pd
import geopandas as gpd
import numpy as np
import os
from datetime import datetime
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

DGA_CSV_PATH = r"\assessment_of_wells_chile\data\DGA\DGA_dataset_analysis_output\DGA_Data_Clean_With_Spatial_Joins.csv"

GDB_PATH = r"\assessment_of_wells_chile\arcgis\assessment_of_wells_chile\Default.gdb"
LAYER_CENSO_2024 = "Censo_24_Merge"

SHP_CUENCAS = r"\assessment_of_wells_chile\data\Basins\Cuencas_BNA\Cuencas_BNA.shp"
SHP_SHAC = r"\assessment_of_wells_chile\data\Aquifers\INV_ACUIFEROS_SHAC_202302\INV_ACUIFEROS_SHAC.shp"

OUTPUT_FOLDER = r"\assessment_of_wells_chile\data\DGA_vs_Census_Comparison"

COL_AREA_24 = 'AREA_C'
COL_PERS_24 = 'n_per'
COL_VIVIENDAS_24 = 'n_hog'
COL_VIVIENDAS_TOTALES_24 = 'n_vp'

COL_POZO_24 = 'n_fuente_agua_pozo'
COL_RED_PUBLICA_24 = 'n_fuente_agua_publica'
COL_CAMION_24 = 'n_fuente_agua_camion'
COL_RIO_24 = 'n_fuente_agua_rio'

COL_DGA_CAUDAL = 'Caudal_Ls'

LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR = 31557.6

WATER_SOURCES = {
    'pozo': {
        'name': 'Pozo o Noria',
        'name_short': 'Pozo',
        'col_2024': COL_POZO_24,
        'description': 'Viviendas con origen del agua por pozo o noria (Wells)',
        'priority': 1
    },
    'red_publica': {
        'name': 'Red Publica',
        'name_short': 'RedPublica',
        'col_2024': COL_RED_PUBLICA_24,
        'description': 'Viviendas con origen del agua por red publica (Public Network)',
        'priority': 2
    },
    'camion': {
        'name': 'Camion Aljibe',
        'name_short': 'Camion',
        'col_2024': COL_CAMION_24,
        'description': 'Viviendas con origen del agua por camion aljibe (Water Truck)',
        'priority': 3
    },
    'rio': {
        'name': 'Rio/Vertiente/Estero',
        'name_short': 'Rio',
        'col_2024': COL_RIO_24,
        'description': 'Viviendas con origen del agua por rio, vertiente, estero, canal, lago, etc. (River/Spring)',
        'priority': 4
    }
}

TARGET_CRS = "EPSG:4326"
SHAC_BUFFER_DISTANCE = 200

REFERENCE_LAYERS = [
    {
        'path': GDB_PATH,
        'layer_name': 'CHL_Municipalities',
        'prefix': 'Muni',
        'name_col': 'NAME',
        'code_col': 'Code_Muni',
        'native_crs': 'EPSG:3857',
        'is_gdb': True
    },
    {
        'path': GDB_PATH,
        'layer_name': 'CHL_Regions',
        'prefix': 'Region',
        'name_col': 'NAME',
        'code_col': 'ID',
        'native_crs': 'EPSG:3857',
        'is_gdb': True
    },
    {
        'path': SHP_CUENCAS,
        'layer_name': None,
        'prefix': 'Cuenca',
        'name_col': 'NOM_CUEN',
        'code_col': 'COD_CUEN',
        'native_crs': 'EPSG:32719',
        'is_gdb': False
    },
    {
        'path': SHP_SHAC,
        'layer_name': None,
        'prefix': 'SHAC',
        'name_col': 'SHAC',
        'code_col': 'COD_SHAC',
        'native_crs': 'EPSG:32719',
        'is_gdb': False
    }
]

def create_output_folder(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    subfolders = ['Shapefiles', 'Excel', 'Reports']
    for f in subfolders:
        Path(os.path.join(path, f)).mkdir(parents=True, exist_ok=True)

def load_reference_layer(layer_config):
    path = layer_config['path']
    prefix = layer_config['prefix']
    native_crs = layer_config['native_crs']
    is_gdb = layer_config['is_gdb']
    layer_name = layer_config.get('layer_name')
    name_col = layer_config['name_col']
    code_col = layer_config['code_col']
    
    try:
        if is_gdb:
            gdf = gpd.read_file(path, layer=layer_name)
        else:
            gdf = gpd.read_file(path)
        
        if gdf.crs is None:
            gdf = gdf.set_crs(native_crs)
        
        if gdf.crs.to_string() != TARGET_CRS:
            gdf = gdf.to_crs(TARGET_CRS)
        
        available_cols = gdf.columns.tolist()
        
        if name_col not in available_cols:
            for col in available_cols:
                if 'name' in col.lower() or 'nom' in col.lower():
                    name_col = col
                    break
        
        if code_col not in available_cols:
            for col in available_cols:
                if 'cod' in col.lower() or 'id' in col.lower():
                    code_col = col
                    break
        
        cols_to_keep = ['geometry']
        if name_col in gdf.columns:
            cols_to_keep.append(name_col)
        if code_col in gdf.columns and code_col != name_col:
            cols_to_keep.append(code_col)
        
        gdf = gdf[cols_to_keep].copy()
        
        rename_dict = {}
        if name_col in gdf.columns:
            rename_dict[name_col] = f'{prefix}_Name'
        if code_col in gdf.columns and code_col != name_col:
            rename_dict[code_col] = f'{prefix}_Code'
        
        gdf = gdf.rename(columns=rename_dict)
        
        return gdf, prefix
        
    except Exception as e:
        print(f"    ERROR loading {prefix}: {str(e)}")
        return None, prefix

def perform_spatial_join(gdf_points, gdf_polygons, prefix):
    if gdf_points is None or len(gdf_points) == 0:
        return gdf_points
    
    if gdf_polygons is None or len(gdf_polygons) == 0:
        return gdf_points
    
    if gdf_points.crs != gdf_polygons.crs:
        gdf_polygons = gdf_polygons.to_crs(gdf_points.crs)
    
    cols_to_drop = [col for col in gdf_points.columns if col.startswith('index_')]
    if cols_to_drop:
        gdf_points = gdf_points.drop(columns=cols_to_drop)
    
    gdf_points = gdf_points.reset_index(drop=True)
    gdf_polygons = gdf_polygons.reset_index(drop=True)
    
    try:
        gdf_joined = gpd.sjoin(
            gdf_points, 
            gdf_polygons, 
            how='left', 
            predicate='within'
        )
        
        cols_to_drop = [col for col in gdf_joined.columns if col.startswith('index_')]
        if cols_to_drop:
            gdf_joined = gdf_joined.drop(columns=cols_to_drop)
        
        gdf_joined = gdf_joined.reset_index(drop=True)
        
        return gdf_joined
        
    except Exception as e:
        print(f"    ERROR in spatial join: {str(e)}")
        return gdf_points

def get_centroid_gdf(gdf):
    gdf_copy = gdf.copy()
    gdf_copy['_original_geometry'] = gdf_copy.geometry
    gdf_copy['geometry'] = gdf_copy.geometry.centroid
    return gdf_copy

def restore_original_geometry(gdf):
    if '_original_geometry' in gdf.columns:
        gdf['geometry'] = gdf['_original_geometry']
        gdf = gdf.drop(columns=['_original_geometry'])
    return gdf

def classify_delta(delta_value, census_wells, dga_rights):
    if census_wells == 0 and dga_rights == 0:
        return "No_Wells_No_Rights"
    elif delta_value > 0:
        return "DGA_Higher_Than_Census"
    elif delta_value < 0:
        return "Census_Higher_Than_DGA"
    else:
        return "DGA_Equals_Census"

def classify_delta_detailed(delta_value, census_wells, dga_rights):
    if census_wells == 0 and dga_rights == 0:
        return "No data - Neither census wells nor DGA rights"
    elif census_wells > 0 and dga_rights == 0:
        return "Census only - Potential unregistered wells"
    elif census_wells == 0 and dga_rights > 0:
        return "DGA only - Rights exist but no census wells"
    elif delta_value > 50:
        return "DGA >> Census - Significant excess of water rights"
    elif delta_value > 10:
        return "DGA > Census - Moderate excess of water rights"
    elif delta_value > 0:
        return "DGA slightly > Census"
    elif delta_value < -50:
        return "Census >> DGA - Many potential unregistered wells"
    elif delta_value < -10:
        return "Census > DGA - Moderate unregistered wells"
    elif delta_value < 0:
        return "Census slightly > DGA - Few potential unregistered"
    else:
        return "DGA equals Census - Full registration"

def classify_primary_water_source(row):
    sources = {
        'RedPublica': row.get('WS_RedPublica', 0),
        'Pozo': row.get('WS_Pozo', 0),
        'Camion': row.get('WS_Camion', 0),
        'Rio': row.get('WS_Rio', 0)
    }
    
    total = sum(sources.values())
    if total == 0:
        return 'No_Data', 0
    
    primary = max(sources, key=sources.get)
    pct = (sources[primary] / total) * 100 if total > 0 else 0
    
    return primary, round(pct, 1)

def buffer_shac_for_spatial_join(gdf_shac, buffer_distance_meters=200):
    original_crs = gdf_shac.crs
    gdf_projected = gdf_shac.to_crs('EPSG:32719')
    gdf_buffered = gdf_projected.copy()
    gdf_buffered['geometry'] = gdf_projected.geometry.buffer(buffer_distance_meters)
    gdf_buffered = gdf_buffered.to_crs(original_crs)
    return gdf_buffered

def convert_ls_to_m3y(caudal_ls):
    return caudal_ls * LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR

def main():
    print("\n" + "="*100)
    print("DGA WATER RIGHTS vs CENSUS 2024 WELLS COMPARISON ANALYSIS")
    print("Including All Water Sources: Red Publica, Pozo, Camion, Rio")
    print("Including DGA Flow Rates (Caudal) Aggregation")
    print("Delta Calculation: DGA Water Rights - Census Wells")
    print("="*100)
    print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    create_output_folder(OUTPUT_FOLDER)
    
    print("\n" + "-"*80)
    print("STEP 1: Loading DGA Water Rights data...")
    print("-"*80)
    
    try:
        df_dga = pd.read_csv(DGA_CSV_PATH)
        print(f"   Loaded {len(df_dga)} DGA water rights records from CSV")
    except Exception as e:
        print(f"   CSV load failed: {e}")
        try:
            excel_path = DGA_CSV_PATH.replace('.csv', '.xlsx')
            df_dga = pd.read_excel(excel_path)
            print(f"   Loaded {len(df_dga)} DGA water rights records from Excel")
        except Exception as e2:
            print(f"   ERROR loading DGA data: {e2}")
            return
    
    df_dga = df_dga[df_dga['lat_wgs84'].notna() & df_dga['lon_wgs84'].notna()].copy()
    df_dga = df_dga[(df_dga['lat_wgs84'] != 0) & (df_dga['lon_wgs84'] != 0)].copy()
    print(f"   Valid DGA records with coordinates: {len(df_dga)}")
    
    if COL_DGA_CAUDAL in df_dga.columns:
        df_dga['Caudal_Ls'] = pd.to_numeric(df_dga[COL_DGA_CAUDAL], errors='coerce').fillna(0)
        total_caudal_ls = df_dga['Caudal_Ls'].sum()
        total_caudal_m3y = convert_ls_to_m3y(total_caudal_ls)
        print(f"   Total DGA Flow Rate (Caudal):")
        print(f"      - {total_caudal_ls:,.2f} L/s")
        print(f"      - {total_caudal_m3y:,.2f} m³/year")
        print(f"   Average Caudal per right: {df_dga['Caudal_Ls'].mean():.2f} L/s")
        print(f"   Max Caudal: {df_dga['Caudal_Ls'].max():.2f} L/s")
        print(f"   Min Caudal (non-zero): {df_dga[df_dga['Caudal_Ls'] > 0]['Caudal_Ls'].min():.4f} L/s")
    else:
        print(f"   WARNING: Column '{COL_DGA_CAUDAL}' not found in DGA data!")
        df_dga['Caudal_Ls'] = 0
    
    gdf_dga = gpd.GeoDataFrame(
        df_dga[['lat_wgs84', 'lon_wgs84', 'Caudal_Ls']],
        geometry=gpd.points_from_xy(df_dga['lon_wgs84'], df_dga['lat_wgs84']),
        crs=TARGET_CRS
    )
    
    print("\n" + "-"*80)
    print("STEP 2: Loading Census 2024 data...")
    print("-"*80)
    
    gdf_census = gpd.read_file(GDB_PATH, layer=LAYER_CENSO_2024)
    original_crs = gdf_census.crs
    if gdf_census.crs != TARGET_CRS:
        gdf_census = gdf_census.to_crs(TARGET_CRS)
    print(f"   Loaded {len(gdf_census)} census blocks")
    print(f"   Original CRS: {original_crs}, Converted to: {TARGET_CRS}")
    
    print("\n   Processing Census 2024 columns...")
    
    print("\n   --- Water Sources ---")
    
    for src_key, src_info in WATER_SOURCES.items():
        col_orig = src_info['col_2024']
        col_new = f"WS_{src_info['name_short']}"
        
        if col_orig in gdf_census.columns:
            gdf_census[col_new] = pd.to_numeric(gdf_census[col_orig], errors='coerce').fillna(0).astype(int)
            total = gdf_census[col_new].sum()
            print(f"   - {src_info['name']} ('{col_orig}'): {total:,} households")
        else:
            gdf_census[col_new] = 0
            print(f"   - WARNING: Column '{col_orig}' not found for {src_info['name']}")
    
    gdf_census['Census_Wells'] = gdf_census['WS_Pozo']
    
    gdf_census['WS_Total'] = (
        gdf_census['WS_RedPublica'] + 
        gdf_census['WS_Pozo'] + 
        gdf_census['WS_Camion'] + 
        gdf_census['WS_Rio']
    )
    
    for src_key, src_info in WATER_SOURCES.items():
        col_ws = f"WS_{src_info['name_short']}"
        col_pct = f"Pct_{src_info['name_short']}"
        gdf_census[col_pct] = np.where(
            gdf_census['WS_Total'] > 0,
            (gdf_census[col_ws] / gdf_census['WS_Total'] * 100).round(2),
            0
        )
    
    primary_sources = gdf_census.apply(classify_primary_water_source, axis=1)
    gdf_census['Primary_WS'] = [x[0] for x in primary_sources]
    gdf_census['Primary_WS_Pct'] = [x[1] for x in primary_sources]
    
    print(f"\n   Primary Water Source Distribution:")
    primary_counts = gdf_census['Primary_WS'].value_counts()
    for src, count in primary_counts.items():
        pct = count / len(gdf_census) * 100
        print(f"      {src}: {count:,} blocks ({pct:.1f}%)")
    
    print("\n   --- Demographics ---")
    
    if COL_PERS_24 in gdf_census.columns:
        gdf_census['Personas_24'] = pd.to_numeric(gdf_census[COL_PERS_24], errors='coerce').fillna(0).astype(int)
        print(f"   - Population: {gdf_census['Personas_24'].sum():,}")
    else:
        gdf_census['Personas_24'] = 0
    
    if COL_VIVIENDAS_24 in gdf_census.columns:
        gdf_census['Viviendas_24'] = pd.to_numeric(gdf_census[COL_VIVIENDAS_24], errors='coerce').fillna(0).astype(int)
        print(f"   - Occupied Households: {gdf_census['Viviendas_24'].sum():,}")
    else:
        gdf_census['Viviendas_24'] = 0
    
    if COL_VIVIENDAS_TOTALES_24 in gdf_census.columns:
        gdf_census['Viv_Total_24'] = pd.to_numeric(gdf_census[COL_VIVIENDAS_TOTALES_24], errors='coerce').fillna(0).astype(int)
        print(f"   - Total Households: {gdf_census['Viv_Total_24'].sum():,}")
    else:
        gdf_census['Viv_Total_24'] = 0
    
    if COL_AREA_24 in gdf_census.columns:
        gdf_census['Is_Rural'] = np.where(
            gdf_census[COL_AREA_24].astype(str).str.upper().str.strip() == 'RURAL', 1, 0
        )
        gdf_census['Area_Type'] = np.where(gdf_census['Is_Rural'] == 1, 'Rural', 'Urban')
        print(f"   - Rural blocks: {gdf_census['Is_Rural'].sum():,}")
        print(f"   - Urban blocks: {(gdf_census['Is_Rural'] == 0).sum():,}")
    else:
        gdf_census['Is_Rural'] = 0
        gdf_census['Area_Type'] = 'Unknown'
    
    print("\n" + "-"*80)
    print("STEP 3: Loading reference layers for spatial joins...")
    print("-"*80)
    
    reference_gdfs = {}
    for layer_config in REFERENCE_LAYERS:
        gdf_ref, prefix = load_reference_layer(layer_config)
        if gdf_ref is not None:
            reference_gdfs[prefix] = gdf_ref
            print(f"   Loaded {prefix}: {len(gdf_ref)} features")
    
    print("\n" + "-"*80)
    print("STEP 4: Counting DGA water rights and summing Caudal per census block...")
    print("-"*80)
    
    if gdf_dga.crs != gdf_census.crs:
        gdf_dga = gdf_dga.to_crs(gdf_census.crs)
    
    gdf_census = gdf_census.reset_index(drop=True)
    gdf_census['block_idx'] = gdf_census.index
    
    print("   Performing spatial join (DGA points -> Census blocks)...")
    gdf_dga_joined = gpd.sjoin(
        gdf_dga[['geometry', 'Caudal_Ls']], 
        gdf_census[['geometry', 'block_idx']], 
        how='left', 
        predicate='within'
    )
    
    print("   Aggregating DGA counts and Caudal per census block...")
    dga_aggregates = gdf_dga_joined.groupby('block_idx').agg(
        DGA_WaterRights=('block_idx', 'count'),
        DGA_Caudal_Ls=('Caudal_Ls', 'sum')
    ).reset_index()
    
    dga_stats = gdf_dga_joined.groupby('block_idx').agg(
        DGA_Caudal_Mean=('Caudal_Ls', 'mean'),
        DGA_Caudal_Max=('Caudal_Ls', 'max'),
        DGA_Caudal_Min=('Caudal_Ls', 'min'),
        DGA_Caudal_Median=('Caudal_Ls', 'median')
    ).reset_index()
    
    dga_aggregates = dga_aggregates.merge(dga_stats, on='block_idx', how='left')
    
    gdf_census = gdf_census.merge(dga_aggregates, on='block_idx', how='left')
    
    gdf_census['DGA_WaterRights'] = gdf_census['DGA_WaterRights'].fillna(0).astype(int)
    gdf_census['DGA_Caudal_Ls'] = gdf_census['DGA_Caudal_Ls'].fillna(0).round(4)
    gdf_census['DGA_Caudal_Mean'] = gdf_census['DGA_Caudal_Mean'].fillna(0).round(4)
    gdf_census['DGA_Caudal_Max'] = gdf_census['DGA_Caudal_Max'].fillna(0).round(4)
    gdf_census['DGA_Caudal_Min'] = gdf_census['DGA_Caudal_Min'].fillna(0).round(4)
    gdf_census['DGA_Caudal_Median'] = gdf_census['DGA_Caudal_Median'].fillna(0).round(4)
    
    gdf_census['DGA_Caudal_m3y'] = (gdf_census['DGA_Caudal_Ls'] * LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR).round(2)
    gdf_census['DGA_Caudal_Mean_m3y'] = (gdf_census['DGA_Caudal_Mean'] * LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR).round(2)
    gdf_census['DGA_Caudal_Max_m3y'] = (gdf_census['DGA_Caudal_Max'] * LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR).round(2)
    gdf_census['DGA_Caudal_Min_m3y'] = (gdf_census['DGA_Caudal_Min'] * LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR).round(2)
    gdf_census['DGA_Caudal_Median_m3y'] = (gdf_census['DGA_Caudal_Median'] * LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR).round(2)
    
    print(f"\n   === DGA SPATIAL JOIN RESULTS ===")
    print(f"   DGA points successfully joined: {gdf_dga_joined['block_idx'].notna().sum():,}")
    print(f"   DGA points outside census blocks: {gdf_dga_joined['block_idx'].isna().sum():,}")
    print(f"   Census blocks with DGA rights: {(gdf_census['DGA_WaterRights'] > 0).sum():,}")
    print(f"   Total DGA rights assigned to blocks: {gdf_census['DGA_WaterRights'].sum():,}")
    
    print(f"\n   === DGA CAUDAL SUMMARY ===")
    print(f"   Total Caudal in census blocks: {gdf_census['DGA_Caudal_Ls'].sum():,.2f} L/s")
    print(f"   Total Caudal in census blocks: {gdf_census['DGA_Caudal_m3y'].sum():,.2f} m³/year")
    print(f"   Blocks with Caudal > 0: {(gdf_census['DGA_Caudal_Ls'] > 0).sum():,}")
    print(f"   Max Caudal in single block: {gdf_census['DGA_Caudal_Ls'].max():,.2f} L/s")
    print(f"   Average Caudal per block (with rights): {gdf_census[gdf_census['DGA_Caudal_Ls'] > 0]['DGA_Caudal_Ls'].mean():,.2f} L/s")
    
    print("\n" + "-"*80)
    print("STEP 5: Calculating Delta (DGA - Census) and Classification...")
    print("-"*80)
    
    gdf_census['Delta_DGA_Census'] = gdf_census['DGA_WaterRights'] - gdf_census['Census_Wells']
    
    gdf_census['Delta_Census_DGA'] = gdf_census['Census_Wells'] - gdf_census['DGA_WaterRights']
    
    gdf_census['Comparison'] = gdf_census.apply(
        lambda row: classify_delta(row['Delta_DGA_Census'], row['Census_Wells'], row['DGA_WaterRights']),
        axis=1
    )
    
    gdf_census['Comparison_Detail'] = gdf_census.apply(
        lambda row: classify_delta_detailed(row['Delta_DGA_Census'], row['Census_Wells'], row['DGA_WaterRights']),
        axis=1
    )
    
    gdf_census['Caudal_per_CensusWell'] = np.where(
        gdf_census['Census_Wells'] > 0,
        (gdf_census['DGA_Caudal_Ls'] / gdf_census['Census_Wells']).round(4),
        0
    )
    
    print(f"\n   === DELTA SUMMARY (DGA - Census) ===")
    print(f"   Total Census Wells (Pozo): {gdf_census['Census_Wells'].sum():,}")
    print(f"   Total DGA Rights: {gdf_census['DGA_WaterRights'].sum():,}")
    print(f"   Total Delta: {gdf_census['Delta_DGA_Census'].sum():,}")
    
    print(f"\n   === WATER SOURCES SUMMARY ===")
    for src_key, src_info in WATER_SOURCES.items():
        col_ws = f"WS_{src_info['name_short']}"
        total = gdf_census[col_ws].sum()
        pct = total / gdf_census['WS_Total'].sum() * 100 if gdf_census['WS_Total'].sum() > 0 else 0
        print(f"   - {src_info['name']}: {total:,} ({pct:.1f}%)")
    
    print(f"\n   === COMPARISON CLASSIFICATION ===")
    comparison_counts = gdf_census['Comparison'].value_counts()
    for comp_type, count in comparison_counts.items():
        pct = count / len(gdf_census) * 100
        print(f"   - {comp_type}: {count:,} blocks ({pct:.1f}%)")
    
    print("\n" + "-"*80)
    print("STEP 6: Spatial joins with reference layers...")
    print("-"*80)
    
    gdf_census_centroids = get_centroid_gdf(gdf_census)
    
    for prefix, ref_gdf in reference_gdfs.items():
        print(f"   Joining with {prefix}...")
        if prefix == 'SHAC':
            ref_gdf_buffered = buffer_shac_for_spatial_join(ref_gdf, SHAC_BUFFER_DISTANCE)
            gdf_census_centroids = perform_spatial_join(gdf_census_centroids, ref_gdf_buffered, prefix)
        else:
            gdf_census_centroids = perform_spatial_join(gdf_census_centroids, ref_gdf, prefix)
    
    gdf_census_joined = restore_original_geometry(gdf_census_centroids)
    
    centroids = gdf_census_joined.geometry.centroid
    gdf_census_joined['Centroid_Lon'] = centroids.x
    gdf_census_joined['Centroid_Lat'] = centroids.y
    
    print("\n" + "-"*80)
    print("STEP 7: Preparing output columns...")
    print("-"*80)
    
    output_cols = [
        'Census_Wells',
        'DGA_WaterRights', 
        'Delta_DGA_Census',
        'Delta_Census_DGA',
        'Comparison',
        'Comparison_Detail',
        
        'DGA_Caudal_Ls',
        'DGA_Caudal_Mean',
        'DGA_Caudal_Median',
        'DGA_Caudal_Max',
        'DGA_Caudal_Min',
        
        'DGA_Caudal_m3y',
        'DGA_Caudal_Mean_m3y',
        'DGA_Caudal_Median_m3y',
        'DGA_Caudal_Max_m3y',
        'DGA_Caudal_Min_m3y',
        
        'Caudal_per_CensusWell',
        
        'WS_RedPublica',
        'WS_Pozo',
        'WS_Camion',
        'WS_Rio',
        'WS_Total',
        
        'Pct_RedPublica',
        'Pct_Pozo',
        'Pct_Camion',
        'Pct_Rio',
        
        'Primary_WS',
        'Primary_WS_Pct',
        
        'Personas_24',
        'Viviendas_24',
        'Viv_Total_24',
        'Area_Type',
        'Is_Rural',
        
        'Muni_Name', 'Muni_Code',
        'Region_Name', 'Region_Code',
        'Cuenca_Name', 'Cuenca_Code',
        'SHAC_Name', 'SHAC_Code',
        
        'Centroid_Lon', 'Centroid_Lat',
        
        'geometry'
    ]
    
    available_cols = [c for c in output_cols if c in gdf_census_joined.columns]
    print(f"   Output columns: {len(available_cols)}")
    
    gdf_output = gdf_census_joined[available_cols].copy()
    
    print("\n" + "-"*80)
    print("STEP 8: Saving Shapefile...")
    print("-"*80)
    
    shp_rename = {
        'Census_Wells': 'Cens_Well',
        'DGA_WaterRights': 'DGA_Rights',
        'Delta_DGA_Census': 'Delta_DGA',
        'Delta_Census_DGA': 'Delta_Cens',
        'Comparison': 'Comparison',
        'Comparison_Detail': 'Comp_Det',
        'DGA_Caudal_Ls': 'DGA_Q_Ls',
        'DGA_Caudal_Mean': 'DGA_Q_Mean',
        'DGA_Caudal_Median': 'DGA_Q_Med',
        'DGA_Caudal_Max': 'DGA_Q_Max',
        'DGA_Caudal_Min': 'DGA_Q_Min',
        'DGA_Caudal_m3y': 'DGA_Q_m3y',
        'DGA_Caudal_Mean_m3y': 'Q_Mean_m3y',
        'DGA_Caudal_Median_m3y': 'Q_Med_m3y',
        'DGA_Caudal_Max_m3y': 'Q_Max_m3y',
        'DGA_Caudal_Min_m3y': 'Q_Min_m3y',
        
        'Caudal_per_CensusWell': 'Q_perWell',
        'WS_RedPublica': 'WS_RedPub',
        'WS_Pozo': 'WS_Pozo',
        'WS_Camion': 'WS_Camion',
        'WS_Rio': 'WS_Rio',
        'WS_Total': 'WS_Total',
        'Pct_RedPublica': 'Pct_RedPub',
        'Pct_Pozo': 'Pct_Pozo',
        'Pct_Camion': 'Pct_Camion',
        'Pct_Rio': 'Pct_Rio',
        'Primary_WS': 'Prim_WS',
        'Primary_WS_Pct': 'Prim_WS_Pc',
        'Personas_24': 'Personas',
        'Viviendas_24': 'Viviendas',
        'Viv_Total_24': 'Viv_Total',
        'Area_Type': 'Area_Type',
        'Is_Rural': 'Is_Rural',
        'Muni_Name': 'Muni_Name',
        'Muni_Code': 'Muni_Code',
        'Region_Name': 'Reg_Name',
        'Region_Code': 'Reg_Code',
        'Cuenca_Name': 'Cuen_Name',
        'Cuenca_Code': 'Cuen_Code',
        'SHAC_Name': 'SHAC_Name',
        'SHAC_Code': 'SHAC_Code',
        'Centroid_Lon': 'Cent_Lon',
        'Centroid_Lat': 'Cent_Lat'
    }
    
    gdf_shp = gdf_output.copy()
    gdf_shp = gdf_shp.rename(columns={k: v for k, v in shp_rename.items() if k in gdf_shp.columns})
    
    shp_path = os.path.join(OUTPUT_FOLDER, 'Shapefiles', 'Census2024_vs_DGA_Comparison.shp')
    gdf_shp.to_file(shp_path, driver='ESRI Shapefile', encoding='utf-8')
    print(f"   Saved: {shp_path}")
    
    print("\n" + "-"*80)
    print("STEP 9: Saving Excel files...")
    print("-"*80)
    
    df_block = gdf_output.drop(columns='geometry').copy()
    block_excel_path = os.path.join(OUTPUT_FOLDER, 'Excel', 'Census2024_vs_DGA_Block_Level.xlsx')
    df_block.to_excel(block_excel_path, index=False)
    print(f"   Saved: {block_excel_path}")
    
    print("\n" + "-"*80)
    print("STEP 10: Calculating aggregated statistics at multiple levels...")
    print("-"*80)
    
    aggregation_levels = [
        ('Region', 'Region_Name', 'Region_Code'),
        ('Municipality', 'Muni_Name', 'Muni_Code'),
        ('Basin', 'Cuenca_Name', 'Cuenca_Code'),
        ('SHAC', 'SHAC_Name', 'SHAC_Code')
    ]
    
    all_stats = {}
    
    for level_name, name_col, code_col in aggregation_levels:
        if name_col not in gdf_output.columns:
            print(f"   Skipping {level_name} - column '{name_col}' not available")
            continue
        
        print(f"   Aggregating at {level_name} level...")
        
        gdf_level = gdf_output[gdf_output[name_col].notna()].copy()
        
        if len(gdf_level) == 0:
            print(f"      No valid data for {level_name}")
            continue
        
        agg_dict = {
            'Census_Wells': 'sum',
            'DGA_WaterRights': 'sum',
            'Delta_DGA_Census': 'sum',
            'Delta_Census_DGA': 'sum',
            'DGA_Caudal_Ls': 'sum',
            'DGA_Caudal_m3y': 'sum',
            'WS_RedPublica': 'sum',
            'WS_Pozo': 'sum',
            'WS_Camion': 'sum',
            'WS_Rio': 'sum',
            'WS_Total': 'sum',
            'Personas_24': 'sum',
            'Viviendas_24': 'sum',
            'Viv_Total_24': 'sum',
            'Is_Rural': 'sum'
        }
        
        agg_dict = {k: v for k, v in agg_dict.items() if k in gdf_level.columns}
        
        groupby_cols = [name_col]
        if code_col in gdf_level.columns:
            groupby_cols.append(code_col)
        
        stats = gdf_level.groupby(groupby_cols, dropna=False).agg(agg_dict).reset_index()
        
        block_counts = gdf_level.groupby(groupby_cols, dropna=False).size().reset_index(name='N_Blocks')
        stats = stats.merge(block_counts, on=groupby_cols)
        
        caudal_stats = gdf_level.groupby(groupby_cols, dropna=False).agg(
            DGA_Caudal_Mean=('DGA_Caudal_Ls', 'mean'),
            DGA_Caudal_Max=('DGA_Caudal_Ls', 'max')
        ).reset_index()
        stats = stats.merge(caudal_stats, on=groupby_cols, how='left')
        stats['DGA_Caudal_Mean'] = stats['DGA_Caudal_Mean'].round(4)
        stats['DGA_Caudal_Max'] = stats['DGA_Caudal_Max'].round(4)
        
        for comp_type in ['Census_Higher_Than_DGA', 'DGA_Higher_Than_Census', 'DGA_Equals_Census', 'No_Wells_No_Rights']:
            comp_counts = gdf_level.groupby(groupby_cols, dropna=False).apply(
                lambda x: (x['Comparison'] == comp_type).sum()
            ).reset_index(name=f'N_{comp_type.replace("_", "")}')
            stats = stats.merge(comp_counts, on=groupby_cols, how='left')
        
        for ws_type in ['RedPublica', 'Pozo', 'Camion', 'Rio', 'No_Data']:
            ws_counts = gdf_level.groupby(groupby_cols, dropna=False).apply(
                lambda x: (x['Primary_WS'] == ws_type).sum()
            ).reset_index(name=f'N_Primary_{ws_type}')
            stats = stats.merge(ws_counts, on=groupby_cols, how='left')
        
        for src_key, src_info in WATER_SOURCES.items():
            col_ws = f"WS_{src_info['name_short']}"
            col_pct = f"Agg_Pct_{src_info['name_short']}"
            if col_ws in stats.columns and 'WS_Total' in stats.columns:
                stats[col_pct] = np.where(
                    stats['WS_Total'] > 0,
                    (stats[col_ws] / stats['WS_Total'] * 100).round(2),
                    0
                )
        
        stats['Pct_CensusWells_WithDGA'] = np.where(
            stats['Census_Wells'] > 0,
            (stats['DGA_WaterRights'] / stats['Census_Wells'] * 100).round(2),
            np.nan
        )
        
        stats['Pct_DGA_WithCensusWells'] = np.where(
            stats['DGA_WaterRights'] > 0,
            (stats['Census_Wells'] / stats['DGA_WaterRights'] * 100).round(2),
            np.nan
        )
        
        stats['Agg_Caudal_per_CensusWell'] = np.where(
            stats['Census_Wells'] > 0,
            (stats['DGA_Caudal_Ls'] / stats['Census_Wells']).round(4),
            0
        )
        
        stats['Agg_Caudal_per_DGARight'] = np.where(
            stats['DGA_WaterRights'] > 0,
            (stats['DGA_Caudal_Ls'] / stats['DGA_WaterRights']).round(4),
            0
        )
        
        stats['Agg_Comparison'] = np.where(
            stats['Delta_DGA_Census'] > 0, 'DGA_Higher',
            np.where(stats['Delta_DGA_Census'] < 0, 'Census_Higher', 'Equal')
        )
        
        def get_dominant_ws(row):
            sources = {
                'RedPublica': row.get('WS_RedPublica', 0),
                'Pozo': row.get('WS_Pozo', 0),
                'Camion': row.get('WS_Camion', 0),
                'Rio': row.get('WS_Rio', 0)
            }
            total = sum(sources.values())
            if total == 0:
                return 'No_Data'
            return max(sources, key=sources.get)
        
        stats['Dominant_WS'] = stats.apply(get_dominant_ws, axis=1)
        
        stats['N_Urban_Blocks'] = stats['N_Blocks'] - stats['Is_Rural']
        stats = stats.rename(columns={'Is_Rural': 'N_Rural_Blocks'})
        
        stats = stats.sort_values('Delta_Census_DGA', ascending=False)
        
        all_stats[level_name] = stats
        print(f"      {level_name}: {len(stats)} units analyzed")
        print(f"         Total Caudal: {stats['DGA_Caudal_Ls'].sum():,.2f} L/s = {stats['DGA_Caudal_m3y'].sum():,.2f} m³/year")
    
    print("\n" + "-"*80)
    print("STEP 11: Saving aggregated Excel files...")
    print("-"*80)
    
    combined_path = os.path.join(OUTPUT_FOLDER, 'Excel', 'Census2024_vs_DGA_All_Levels.xlsx')
    
    with pd.ExcelWriter(combined_path, engine='openpyxl') as writer:
        
        total_census = gdf_output['Census_Wells'].sum()
        total_dga = gdf_output['DGA_WaterRights'].sum()
        total_caudal_ls = gdf_output['DGA_Caudal_Ls'].sum()
        total_caudal_m3y = gdf_output['DGA_Caudal_m3y'].sum()
        
        ws_totals = {src_info['name']: gdf_output[f"WS_{src_info['name_short']}"].sum() 
                     for src_key, src_info in WATER_SOURCES.items()}
        ws_grand_total = sum(ws_totals.values())
        
        national_summary = pd.DataFrame({
            'Metric': [
                'Total Census Blocks Analyzed',
                'Total Census Wells (Households with Pozo)',
                'Total DGA Water Rights (in census blocks)',
                'Delta (DGA - Census)',
                'Delta (Census - DGA)',
                'DGA Coverage Rate (DGA/Census %)',
                '',
                '--- DGA FLOW RATES (CAUDAL) ---',
                'Total Caudal (L/s)',
                'Total Caudal (m³/year)',
                'Average Caudal per DGA Right (L/s)',
                'Average Caudal per Census Well (L/s)',
                'Max Caudal in single block (L/s)',
                'Blocks with Caudal > 0',
                '',
                '--- WATER SOURCES ---',
                'Red Publica (Public Network)',
                'Pozo o Noria (Wells)',
                'Camion Aljibe (Water Truck)',
                'Rio/Vertiente/Estero (River/Spring)',
                'Total Water Source Records',
                '',
                '--- WATER SOURCE PERCENTAGES ---',
                '% Red Publica',
                '% Pozo',
                '% Camion',
                '% Rio',
                '',
                '--- COMPARISON CLASSIFICATION ---',
                'Blocks: Census > DGA (potential unregistered)',
                'Blocks: DGA > Census', 
                'Blocks: DGA = Census',
                'Blocks: No Wells, No Rights',
                '',
                '--- DEMOGRAPHICS ---',
                'Total Rural Blocks',
                'Total Urban Blocks',
                'Total Population',
                'Total Households (Occupied)',
                'Total Households (All)'
            ],
            'Value': [
                len(gdf_output),
                total_census,
                total_dga,
                total_dga - total_census,
                total_census - total_dga,
                (total_dga / total_census * 100) if total_census > 0 else 0,
                '',
                '',
                total_caudal_ls,
                total_caudal_m3y,
                (total_caudal_ls / total_dga) if total_dga > 0 else 0,
                (total_caudal_ls / total_census) if total_census > 0 else 0,
                gdf_output['DGA_Caudal_Ls'].max(),
                (gdf_output['DGA_Caudal_Ls'] > 0).sum(),
                '',
                '',
                ws_totals['Red Publica'],
                ws_totals['Pozo o Noria'],
                ws_totals['Camion Aljibe'],
                ws_totals['Rio/Vertiente/Estero'],
                ws_grand_total,
                '',
                '',
                (ws_totals['Red Publica'] / ws_grand_total * 100) if ws_grand_total > 0 else 0,
                (ws_totals['Pozo o Noria'] / ws_grand_total * 100) if ws_grand_total > 0 else 0,
                (ws_totals['Camion Aljibe'] / ws_grand_total * 100) if ws_grand_total > 0 else 0,
                (ws_totals['Rio/Vertiente/Estero'] / ws_grand_total * 100) if ws_grand_total > 0 else 0,
                '',
                '',
                (gdf_output['Comparison'] == 'Census_Higher_Than_DGA').sum(),
                (gdf_output['Comparison'] == 'DGA_Higher_Than_Census').sum(),
                (gdf_output['Comparison'] == 'DGA_Equals_Census').sum(),
                (gdf_output['Comparison'] == 'No_Wells_No_Rights').sum(),
                '',
                '',
                gdf_output['Is_Rural'].sum(),
                len(gdf_output) - gdf_output['Is_Rural'].sum(),
                gdf_output['Personas_24'].sum(),
                gdf_output['Viviendas_24'].sum(),
                gdf_output['Viv_Total_24'].sum()
            ]
        })
        national_summary.to_excel(writer, sheet_name='National_Summary', index=False)
        
        df_block_sample = df_block.head(50000)
        df_block_sample.to_excel(writer, sheet_name='Block_Level_Sample', index=False)
        
        for level_name, stats in all_stats.items():
            sheet_name = f'{level_name}_Level'
            stats.to_excel(writer, sheet_name=sheet_name, index=False)
        
        primary_ws_summary = gdf_output.groupby('Primary_WS').agg({
            'Census_Wells': 'sum',
            'DGA_WaterRights': 'sum',
            'DGA_Caudal_Ls': 'sum',
            'DGA_Caudal_m3y': 'sum',
            'WS_RedPublica': 'sum',
            'WS_Pozo': 'sum',
            'WS_Camion': 'sum',
            'WS_Rio': 'sum',
            'Personas_24': 'sum',
            'Viviendas_24': 'sum'
        }).reset_index()
        primary_ws_summary['N_Blocks'] = gdf_output.groupby('Primary_WS').size().values
        primary_ws_summary.to_excel(writer, sheet_name='By_Primary_WaterSource', index=False)
        
        top_caudal_blocks = df_block.nlargest(100, 'DGA_Caudal_Ls')
        top_caudal_blocks.to_excel(writer, sheet_name='Top_100_Blocks_by_Caudal', index=False)
        
        interpretation = pd.DataFrame({
            'Column': [
                '--- COMPARISON COLUMNS ---',
                'Census_Wells',
                'DGA_WaterRights',
                'Delta_DGA_Census',
                'Delta_Census_DGA',
                'Comparison',
                '',
                '--- DGA CAUDAL (FLOW RATE) COLUMNS ---',
                'DGA_Caudal_Ls',
                'DGA_Caudal_m3y',
                'DGA_Caudal_Mean',
                'DGA_Caudal_Median',
                'DGA_Caudal_Max',
                'DGA_Caudal_Min',
                'DGA_Caudal_Mean_m3y',
                'DGA_Caudal_Median_m3y',
                'DGA_Caudal_Max_m3y',
                'DGA_Caudal_Min_m3y',
                'Caudal_per_CensusWell',
                '',
                '--- WATER SOURCE COLUMNS ---',
                'WS_RedPublica',
                'WS_Pozo',
                'WS_Camion',
                'WS_Rio',
                'WS_Total',
                'Primary_WS',
                'Primary_WS_Pct',
                '',
                '--- CONVERSION ---',
                'L/s to m³/year'
            ],
            'Description': [
                '',
                'Number of households reporting well (Pozo) as water source in Census 2024',
                'Number of DGA water rights (points) within census block',
                'DGA Water Rights MINUS Census Wells',
                'Census Wells MINUS DGA Water Rights',
                'Classification of comparison result',
                '',
                '',
                'Sum of DGA water rights flow rates in Liters per Second',
                'Sum of DGA water rights flow rates in Cubic Meters per Year',
                'Average flow rate of DGA rights in the block (L/s)',
                'Median flow rate of DGA rights in the block (L/s)',
                'Maximum flow rate of DGA rights in the block (L/s)',
                'Minimum flow rate of DGA rights in the block (L/s)',
                'Average flow rate of DGA rights in the block (m3/year)',
                'Median flow rate of DGA rights in the block (m3/year)',
                'Maximum flow rate of DGA rights in the block (m3/year)',
                'Minimum flow rate of DGA rights in the block (m3/year)',
                'Total Caudal divided by number of Census Wells (L/s per well)',
                '',
                '',
                'Households with public water network connection',
                'Households with well (pozo) as water source',
                'Households supplied by water truck (camion aljibe)',
                'Households using river/spring/stream water',
                'Total households with water source data',
                'Dominant water source type in census block',
                'Percentage of households using primary water source',
                '',
                '',
                f'1 L/s = {LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR:,.2f} m³/year'
            ]
        })
        interpretation.to_excel(writer, sheet_name='Column_Definitions', index=False)
    
    print(f"   Saved: {combined_path}")
    
    for level_name, stats in all_stats.items():
        level_path = os.path.join(OUTPUT_FOLDER, 'Excel', f'Census2024_vs_DGA_{level_name}_Level.xlsx')
        
        with pd.ExcelWriter(level_path, engine='openpyxl') as writer:
            stats.to_excel(writer, sheet_name='All_Data', index=False)
            
            census_higher = stats[stats['Agg_Comparison'] == 'Census_Higher'].copy()
            census_higher = census_higher.sort_values('Delta_Census_DGA', ascending=False)
            census_higher.to_excel(writer, sheet_name='Census_Higher_Than_DGA', index=False)
            
            dga_higher = stats[stats['Agg_Comparison'] == 'DGA_Higher'].copy()
            dga_higher = dga_higher.sort_values('Delta_DGA_Census', ascending=False)
            dga_higher.to_excel(writer, sheet_name='DGA_Higher_Than_Census', index=False)
            
            top_caudal = stats.nlargest(20, 'DGA_Caudal_Ls')
            top_caudal.to_excel(writer, sheet_name='Top_20_by_Caudal', index=False)
            
            by_dominant_ws = stats.groupby('Dominant_WS').agg({
                'N_Blocks': 'sum',
                'Census_Wells': 'sum',
                'DGA_WaterRights': 'sum',
                'DGA_Caudal_Ls': 'sum',
                'DGA_Caudal_m3y': 'sum',
                'Delta_Census_DGA': 'sum',
                'WS_RedPublica': 'sum',
                'WS_Pozo': 'sum',
                'WS_Camion': 'sum',
                'WS_Rio': 'sum',
                'Personas_24': 'sum'
            }).reset_index()
            by_dominant_ws.to_excel(writer, sheet_name='By_Dominant_WaterSource', index=False)
            
            name_col = [c for c in stats.columns if c.endswith('_Name')][0]
            level_summary = pd.DataFrame({
                'Metric': [
                    f'Total {level_name}s',
                    'Census Wells Total',
                    'DGA Rights Total',
                    'Net Delta (Census - DGA)',
                    f'{level_name}s with Census > DGA',
                    f'{level_name}s with DGA > Census',
                    f'{level_name}s with Equal',
                    '',
                    '--- DGA CAUDAL ---',
                    'Total Caudal (L/s)',
                    'Total Caudal (m³/year)',
                    'Average Caudal per DGA Right (L/s)',
                    f'{level_name} with highest Caudal',
                    'Highest Caudal value (L/s)',
                    '',
                    '--- WATER SOURCES ---',
                    'Red Publica Total',
                    'Pozo Total',
                    'Camion Total',
                    'Rio Total',
                    '',
                    '--- DOMINANT WATER SOURCE ---',
                    f'{level_name}s with RedPublica dominant',
                    f'{level_name}s with Pozo dominant',
                    f'{level_name}s with Camion dominant',
                    f'{level_name}s with Rio dominant'
                ],
                'Value': [
                    len(stats),
                    stats['Census_Wells'].sum(),
                    stats['DGA_WaterRights'].sum(),
                    stats['Delta_Census_DGA'].sum(),
                    (stats['Agg_Comparison'] == 'Census_Higher').sum(),
                    (stats['Agg_Comparison'] == 'DGA_Higher').sum(),
                    (stats['Agg_Comparison'] == 'Equal').sum(),
                    '',
                    '',
                    stats['DGA_Caudal_Ls'].sum(),
                    stats['DGA_Caudal_m3y'].sum(),
                    (stats['DGA_Caudal_Ls'].sum() / stats['DGA_WaterRights'].sum()) if stats['DGA_WaterRights'].sum() > 0 else 0,
                    stats.loc[stats['DGA_Caudal_Ls'].idxmax(), name_col] if len(stats) > 0 else 'N/A',
                    stats['DGA_Caudal_Ls'].max(),
                    '',
                    '',
                    stats['WS_RedPublica'].sum(),
                    stats['WS_Pozo'].sum(),
                    stats['WS_Camion'].sum(),
                    stats['WS_Rio'].sum(),
                    '',
                    '',
                    (stats['Dominant_WS'] == 'RedPublica').sum(),
                    (stats['Dominant_WS'] == 'Pozo').sum(),
                    (stats['Dominant_WS'] == 'Camion').sum(),
                    (stats['Dominant_WS'] == 'Rio').sum()
                ]
            })
            level_summary.to_excel(writer, sheet_name='Summary', index=False)
        
        print(f"   Saved: {level_path}")
    
    print("\n" + "-"*80)
    print("STEP 12: Generating text report...")
    print("-"*80)
    
    report_lines = []
    report_lines.append("="*100)
    report_lines.append("DGA WATER RIGHTS vs CENSUS 2024 WELLS COMPARISON REPORT")
    report_lines.append("Including All Water Sources and DGA Flow Rates (Caudal)")
    report_lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report_lines.append("="*100)
    
    report_lines.append("\nDELTA CALCULATION:")
    report_lines.append("  Delta = DGA Water Rights - Census Wells (Pozo)")
    report_lines.append("  Positive Delta: More DGA rights than census wells")
    report_lines.append("  Negative Delta: More census wells than DGA rights (potential unregistered)")
    
    report_lines.append("\nCAUDAL CONVERSION:")
    report_lines.append(f"  1 L/s = {LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR:,.2f} m³/year")
    
    report_lines.append("\n" + "="*80)
    report_lines.append("NATIONAL SUMMARY")
    report_lines.append("="*80)
    report_lines.append(f"Total Census Blocks: {len(gdf_output):,}")
    report_lines.append(f"Total Population: {gdf_output['Personas_24'].sum():,}")
    report_lines.append(f"Total Households: {gdf_output['Viviendas_24'].sum():,}")
    
    report_lines.append("\n--- DGA vs CENSUS COMPARISON ---")
    report_lines.append(f"Total Census Wells (Pozo): {gdf_output['Census_Wells'].sum():,}")
    report_lines.append(f"Total DGA Water Rights: {gdf_output['DGA_WaterRights'].sum():,}")
    report_lines.append(f"Delta (DGA - Census): {gdf_output['Delta_DGA_Census'].sum():,}")
    report_lines.append(f"Delta (Census - DGA): {gdf_output['Delta_Census_DGA'].sum():,}")
    
    coverage_rate = (gdf_output['DGA_WaterRights'].sum() / gdf_output['Census_Wells'].sum() * 100) if gdf_output['Census_Wells'].sum() > 0 else 0
    report_lines.append(f"DGA Coverage Rate: {coverage_rate:.2f}%")
    
    report_lines.append("\n--- DGA FLOW RATES (CAUDAL) ---")
    total_caudal_ls = gdf_output['DGA_Caudal_Ls'].sum()
    total_caudal_m3y = gdf_output['DGA_Caudal_m3y'].sum()
    report_lines.append(f"Total Caudal: {total_caudal_ls:,.2f} L/s")
    report_lines.append(f"Total Caudal: {total_caudal_m3y:,.2f} m³/year")
    report_lines.append(f"Average Caudal per DGA Right: {(total_caudal_ls / gdf_output['DGA_WaterRights'].sum()) if gdf_output['DGA_WaterRights'].sum() > 0 else 0:.2f} L/s")
    report_lines.append(f"Average Caudal per Census Well: {(total_caudal_ls / gdf_output['Census_Wells'].sum()) if gdf_output['Census_Wells'].sum() > 0 else 0:.2f} L/s")
    report_lines.append(f"Max Caudal in single block: {gdf_output['DGA_Caudal_Ls'].max():,.2f} L/s")
    report_lines.append(f"Blocks with Caudal > 0: {(gdf_output['DGA_Caudal_Ls'] > 0).sum():,}")
    
    report_lines.append("\n--- WATER SOURCES SUMMARY ---")
    report_lines.append(f"{'Source':<30} {'Households':>15} {'Percentage':>12}")
    report_lines.append("-"*60)
    ws_grand_total = gdf_output['WS_Total'].sum()
    for src_key, src_info in WATER_SOURCES.items():
        col_ws = f"WS_{src_info['name_short']}"
        total = gdf_output[col_ws].sum()
        pct = (total / ws_grand_total * 100) if ws_grand_total > 0 else 0
        report_lines.append(f"{src_info['name']:<30} {total:>15,} {pct:>11.2f}%")
    report_lines.append(f"{'TOTAL':<30} {ws_grand_total:>15,} {'100.00':>11}%")
    
    report_lines.append("\n--- PRIMARY WATER SOURCE BY BLOCKS ---")
    primary_counts = gdf_output['Primary_WS'].value_counts()
    for src, count in primary_counts.items():
        pct = count / len(gdf_output) * 100
        report_lines.append(f"  {src}: {count:,} blocks ({pct:.1f}%)")
    
    report_lines.append("\n--- COMPARISON CLASSIFICATION ---")
    for comp_type in ['Census_Higher_Than_DGA', 'DGA_Higher_Than_Census', 'DGA_Equals_Census', 'No_Wells_No_Rights']:
        count = (gdf_output['Comparison'] == comp_type).sum()
        pct = count / len(gdf_output) * 100
        report_lines.append(f"  {comp_type}: {count:,} blocks ({pct:.1f}%)")
    
    for level_name, stats in all_stats.items():
        name_col = [c for c in stats.columns if c.endswith('_Name')][0]
        
        report_lines.append(f"\n" + "="*80)
        report_lines.append(f"{level_name.upper()} LEVEL ANALYSIS")
        report_lines.append("="*80)
        report_lines.append(f"Total units: {len(stats)}")
        report_lines.append(f"Total Census Wells: {stats['Census_Wells'].sum():,}")
        report_lines.append(f"Total DGA Rights: {stats['DGA_WaterRights'].sum():,}")
        report_lines.append(f"Net Delta (Census - DGA): {stats['Delta_Census_DGA'].sum():,}")
        
        report_lines.append(f"\n--- DGA CAUDAL ---")
        report_lines.append(f"  Total Caudal: {stats['DGA_Caudal_Ls'].sum():,.2f} L/s = {stats['DGA_Caudal_m3y'].sum():,.2f} m³/year")
        
        report_lines.append(f"\n--- WATER SOURCES ---")
        report_lines.append(f"  Red Publica: {stats['WS_RedPublica'].sum():,}")
        report_lines.append(f"  Pozo: {stats['WS_Pozo'].sum():,}")
        report_lines.append(f"  Camion: {stats['WS_Camion'].sum():,}")
        report_lines.append(f"  Rio: {stats['WS_Rio'].sum():,}")
        
        report_lines.append(f"\n--- TOP 10: HIGHEST CAUDAL (FLOW RATE) ---")
        top_caudal = stats.nlargest(10, 'DGA_Caudal_Ls')
        report_lines.append(f"{'Rank':<5} {level_name:<35} {'Caudal L/s':>15} {'Caudal m³/y':>18} {'DGA Rights':>12}")
        report_lines.append("-"*90)
        for i, (_, row) in enumerate(top_caudal.iterrows(), 1):
            name = str(row[name_col])[:33]
            report_lines.append(f"{i:<5} {name:<35} {row['DGA_Caudal_Ls']:>15,.2f} {row['DGA_Caudal_m3y']:>18,.2f} {row['DGA_WaterRights']:>12,.0f}")
        
        report_lines.append(f"\n--- TOP 10: HIGHEST CENSUS > DGA (Potential Unregistered Wells) ---")
        top_census = stats[stats['Agg_Comparison'] == 'Census_Higher'].nlargest(10, 'Delta_Census_DGA')
        report_lines.append(f"{'Rank':<5} {level_name:<35} {'Census':>10} {'DGA':>10} {'Delta':>10} {'Caudal L/s':>12}")
        report_lines.append("-"*90)
        for i, (_, row) in enumerate(top_census.iterrows(), 1):
            name = str(row[name_col])[:33]
            report_lines.append(f"{i:<5} {name:<35} {row['Census_Wells']:>10,.0f} {row['DGA_WaterRights']:>10,.0f} {row['Delta_Census_DGA']:>+10,.0f} {row['DGA_Caudal_Ls']:>12,.2f}")
    
    report_lines.append("\n" + "="*100)
    report_lines.append("OUTPUT FILES GENERATED:")
    report_lines.append("="*100)
    report_lines.append(f"Shapefile: {os.path.join(OUTPUT_FOLDER, 'Shapefiles', 'Census2024_vs_DGA_Comparison.shp')}")
    report_lines.append(f"Excel (All Levels): {combined_path}")
    report_lines.append(f"Excel (Block Level): {block_excel_path}")
    for level_name in all_stats.keys():
        report_lines.append(f"Excel ({level_name}): {os.path.join(OUTPUT_FOLDER, 'Excel', f'Census2024_vs_DGA_{level_name}_Level.xlsx')}")
    
    report_lines.append("\n" + "="*100)
    report_lines.append("SHAPEFILE COLUMNS:")
    report_lines.append("="*100)
    report_lines.append("Comparison: Cens_Well, DGA_Rights, Delta_DGA, Delta_Cens, Comparison")
    report_lines.append("Caudal: DGA_Q_Ls (L/s), DGA_Q_m3y (m³/year), DGA_Q_Mean, DGA_Q_Max, DGA_Q_Min, Q_perWell")
    report_lines.append("Water Sources: WS_RedPub, WS_Pozo, WS_Camion, WS_Rio, WS_Total")
    report_lines.append("Percentages: Pct_RedPub, Pct_Pozo, Pct_Camion, Pct_Rio")
    report_lines.append("Primary: Prim_WS, Prim_WS_Pc")
    report_lines.append("Demographics: Personas, Viviendas, Viv_Total, Area_Type, Is_Rural")
    report_lines.append("Spatial: Muni_Name, Muni_Code, Reg_Name, Reg_Code, Cuen_Name, Cuen_Code, SHAC_Name, SHAC_Code")
    
    report_lines.append("\n" + "="*100)
    report_lines.append("END OF REPORT")
    report_lines.append("="*100)
    
    report_path = os.path.join(OUTPUT_FOLDER, 'Reports', f'DGA_vs_Census_Report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt')
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_lines))
    print(f"   Saved: {report_path}")
    
    print("\n" + "\n".join(report_lines[:100]))
    
    print("\n" + "="*100)
    print("ANALYSIS COMPLETE!")
    print("="*100)
    print(f"\nOutput folder: {OUTPUT_FOLDER}")
    print(f"\nFiles generated:")
    print(f"  - Shapefile: Shapefiles/Census2024_vs_DGA_Comparison.shp")
    print(f"  - Excel (All Levels): Excel/Census2024_vs_DGA_All_Levels.xlsx")
    print(f"  - Excel (Block Level): Excel/Census2024_vs_DGA_Block_Level.xlsx")
    for level_name in all_stats.keys():
        print(f"  - Excel ({level_name}): Excel/Census2024_vs_DGA_{level_name}_Level.xlsx")
    print(f"  - Report: Reports/DGA_vs_Census_Report_*.txt")
    
    print(f"\nFinished: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if __name__ == "__main__":
    main()

# Water Consumption Estimation

In [ ]:
#!/usr/bin/env python3

import pandas as pd
import geopandas as gpd
import numpy as np
import os
import warnings
from datetime import datetime
from pathlib import Path

warnings.filterwarnings('ignore')

INPUT_SHAPEFILE = r"\assessment_of_wells_chile\data\DGA_vs_Census_Comparison\Shapefiles\Census2024_vs_DGA_Comparison.shp"

OUTPUT_FOLDER = r"\assessment_of_wells_chile\data\Groundwater_Extraction_Estimates"

C_PC_DOMESTIC = 150
C_PC_ENHANCED = 250
H_AVG_DEFAULT = 3.1

P_PRODUCTIVE = 0.25
Q_PRODUCTIVE_LS = 1.0

SECONDS_PER_DAY = 86400
SECONDS_PER_YEAR = 86400 * 365
LITERS_TO_M3 = 0.001
LITERS_PER_SECOND_TO_CUBIC_METERS_PER_YEAR = 31557.6

SENSITIVITY_PARAMS = {
    'C_pc': {'min': 105, 'base': 150, 'max': 195},
    'p_productive': {'min': 0.15, 'base': 0.25, 'max': 0.35},
    'q_productive': {'min': 0.5, 'base': 1.0, 'max': 1.5}
}

SHP_COL_MAP = {
    'Cens_Well': 'Census_Wells',
    'DGA_Rights': 'DGA_WaterRights',
    'Delta_DGA': 'Delta_DGA_Census',
    'Delta_Cens': 'Delta_Census_DGA',
    'Comparison': 'Comparison',
    'Comp_Det': 'Comparison_Detail',
    'DGA_Q_Ls': 'DGA_Caudal_Ls',
    'DGA_Q_Mean': 'DGA_Caudal_Mean',
    'DGA_Q_Med': 'DGA_Caudal_Median',
    'DGA_Q_Max': 'DGA_Caudal_Max',
    'DGA_Q_Min': 'DGA_Caudal_Min',
    'DGA_Q_m3y': 'DGA_Caudal_m3y',
    'Q_Mean_m3y': 'DGA_Caudal_Mean_m3y',
    'Q_Med_m3y': 'DGA_Caudal_Median_m3y',
    'Q_Max_m3y': 'DGA_Caudal_Max_m3y',
    'Q_Min_m3y': 'DGA_Caudal_Min_m3y',
    'Q_perWell': 'Caudal_per_CensusWell',
    'WS_RedPub': 'WS_RedPublica',
    'WS_Pozo': 'WS_Pozo',
    'WS_Camion': 'WS_Camion',
    'WS_Rio': 'WS_Rio',
    'WS_Total': 'WS_Total',
    'Pct_RedPub': 'Pct_RedPublica',
    'Pct_Pozo': 'Pct_Pozo',
    'Pct_Camion': 'Pct_Camion',
    'Pct_Rio': 'Pct_Rio',
    'Prim_WS': 'Primary_WS',
    'Prim_WS_Pc': 'Primary_WS_Pct',
    'Personas': 'Personas_24',
    'Viviendas': 'Viviendas_24',
    'Viv_Total': 'Viv_Total_24',
    'Area_Type': 'Area_Type',
    'Is_Rural': 'Is_Rural',
    'Muni_Name': 'Muni_Name',
    'Muni_Code': 'Muni_Code',
    'Reg_Name': 'Region_Name',
    'Reg_Code': 'Region_Code',
    'Cuen_Name': 'Cuenca_Name',
    'Cuen_Code': 'Cuenca_Code',
    'SHAC_Name': 'SHAC_Name',
    'SHAC_Code': 'SHAC_Code',
    'Cent_Lon': 'Centroid_Lon',
    'Cent_Lat': 'Centroid_Lat'
}

REVERSE_COL_MAP = {v: k for k, v in SHP_COL_MAP.items()}

def create_output_folder(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    subfolders = ['Shapefiles', 'CSV', 'Excel', 'Reports']
    for f in subfolders:
        Path(os.path.join(path, f)).mkdir(parents=True, exist_ok=True)

def rename_shapefile_columns(gdf):
    rename_dict = {}
    for shp_col, full_col in SHP_COL_MAP.items():
        if shp_col in gdf.columns:
            rename_dict[shp_col] = full_col
    return gdf.rename(columns=rename_dict)

def prepare_shapefile_columns(gdf):
    rename_dict = {}
    for col in gdf.columns:
        if col == 'geometry':
            continue
        if col in REVERSE_COL_MAP:
            rename_dict[col] = REVERSE_COL_MAP[col]
        elif len(col) > 10:
            rename_dict[col] = col[:10]
    return gdf.rename(columns=rename_dict)

def main():
    print("\n" + "="*100)
    print("GROUNDWATER EXTRACTION ESTIMATION FROM CENSUS vs DGA COMPARISON")
    print("Using Pre-Processed Shapefile from Code 1")
    print("="*100)
    print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    create_output_folder(OUTPUT_FOLDER)
    
    print("\n" + "-"*80)
    print("STEP 1: Loading shapefile from Code 1...")
    print("-"*80)
    
    if not os.path.exists(INPUT_SHAPEFILE):
        print(f"   ERROR: Input shapefile not found: {INPUT_SHAPEFILE}")
        print("   Please run Code 1 first to generate the shapefile.")
        return
    
    gdf = gpd.read_file(INPUT_SHAPEFILE)
    print(f"   ✅ Loaded {len(gdf):,} census blocks from shapefile")
    print(f"   CRS: {gdf.crs}")
    print(f"   Original columns: {list(gdf.columns)}")
    
    gdf = rename_shapefile_columns(gdf)
    print(f"   Renamed columns: {list(gdf.columns)}")
    
    print("\n" + "-"*80)
    print("STEP 2: Validating and preparing data...")
    print("-"*80)
    
    required_cols = ['Census_Wells', 'DGA_WaterRights', 'Personas_24', 'Viviendas_24', 'Is_Rural']
    missing_cols = [c for c in required_cols if c not in gdf.columns]
    
    if missing_cols:
        print(f"   ⚠️ Missing columns: {missing_cols}")
        for col in missing_cols:
            for existing_col in gdf.columns:
                if col.lower() in existing_col.lower():
                    print(f"      Found alternative: {existing_col} for {col}")
    
    numeric_cols = ['Census_Wells', 'DGA_WaterRights', 'Personas_24', 'Viviendas_24', 
                    'Is_Rural', 'WS_RedPublica', 'WS_Pozo', 'WS_Camion', 'WS_Rio',
                    'DGA_Caudal_Ls', 'DGA_Caudal_m3y']
    
    for col in numeric_cols:
        if col in gdf.columns:
            gdf[col] = pd.to_numeric(gdf[col], errors='coerce').fillna(0)
    
    print(f"\n   === DATA SUMMARY ===")
    print(f"   Total census blocks: {len(gdf):,}")
    print(f"   Total Census Wells: {gdf['Census_Wells'].sum():,.0f}")
    print(f"   Total DGA Rights: {gdf['DGA_WaterRights'].sum():,.0f}")
    print(f"   Total Population: {gdf['Personas_24'].sum():,.0f}")
    print(f"   Total Households: {gdf['Viviendas_24'].sum():,.0f}")
    print(f"   Rural blocks: {gdf['Is_Rural'].sum():,.0f}")
    print(f"   Urban blocks: {(gdf['Is_Rural'] == 0).sum():,.0f}")
    
    if 'DGA_Caudal_Ls' in gdf.columns:
        print(f"   Total DGA Caudal: {gdf['DGA_Caudal_Ls'].sum():,.2f} L/s")
    
    print("\n" + "-"*80)
    print("STEP 3: Calculating household size (H_avg)...")
    print("-"*80)
    
    gdf['H_Block'] = np.where(
        gdf['Viviendas_24'] > 0,
        gdf['Personas_24'] / gdf['Viviendas_24'],
        H_AVG_DEFAULT
    )
    
    gdf['Used_Default_H'] = np.where(gdf['Viviendas_24'] > 0, 0, 1)
    
    total_personas = gdf['Personas_24'].sum()
    total_viviendas = gdf['Viviendas_24'].sum()
    H_AVG_NATIONAL = total_personas / total_viviendas if total_viviendas > 0 else H_AVG_DEFAULT
    
    blocks_with_real_h = (gdf['Used_Default_H'] == 0).sum()
    blocks_with_default_h = (gdf['Used_Default_H'] == 1).sum()
    pct_real_h = (blocks_with_real_h / len(gdf)) * 100
    
    print(f"   National average H_avg: {H_AVG_NATIONAL:.2f} persons/household")
    print(f"   Blocks with calculated H_avg: {blocks_with_real_h:,} ({pct_real_h:.1f}%)")
    print(f"   Blocks using default ({H_AVG_DEFAULT}): {blocks_with_default_h:,}")
    
    print("\n" + "-"*80)
    print("STEP 4: Calculating discrepancy (unregistered wells)...")
    print("-"*80)
    
    gdf['Discrepancy'] = (gdf['Census_Wells'] - gdf['DGA_WaterRights']).clip(lower=0)
    
    gdf['Ratio_Census_DGA'] = np.where(
        gdf['DGA_WaterRights'] > 0,
        gdf['Census_Wells'] / gdf['DGA_WaterRights'],
        np.nan
    )
    
    total_discrepancy = gdf['Discrepancy'].sum()
    total_census = gdf['Census_Wells'].sum()
    total_dga = gdf['DGA_WaterRights'].sum()
    
    print(f"   Total Census Wells: {total_census:,.0f}")
    print(f"   Total DGA Rights: {total_dga:,.0f}")
    print(f"   Total Discrepancy (potential unregistered): {total_discrepancy:,.0f}")
    print(f"   Ratio Census/DGA: {total_census/total_dga:.2f}x" if total_dga > 0 else "   Ratio: N/A")
    
    print("\n" + "-"*80)
    print("STEP 5: Calculating exclusive well dependence (p_exclusive)...")
    print("-"*80)
    
    if 'WS_RedPublica' in gdf.columns:
        gdf['Pozos_Exclusivos'] = (gdf['Census_Wells'] - gdf['WS_RedPublica']).clip(lower=0)
    else:
        gdf['Pozos_Exclusivos'] = gdf['Census_Wells']
    
    gdf['p_exclusive'] = np.where(
        gdf['Census_Wells'] > 0,
        gdf['Pozos_Exclusivos'] / gdf['Census_Wells'],
        1.0
    )
    gdf['p_exclusive'] = gdf['p_exclusive'].clip(0, 1)
    
    p_exclusive_national = gdf['Pozos_Exclusivos'].sum() / gdf['Census_Wells'].sum() if gdf['Census_Wells'].sum() > 0 else 1.0
    
    print(f"   National p_exclusive: {p_exclusive_national:.2%}")
    print(f"   Meaning: {p_exclusive_national*100:.1f}% of well users have NO public network access")
    
    print("\n" + "-"*80)
    print("STEP 6: Calculating rural proportion (p_rural)...")
    print("-"*80)
    
    gdf['p_rural'] = gdf['Is_Rural'].astype(float)
    
    rural_wells = (gdf['Census_Wells'] * gdf['Is_Rural']).sum()
    total_wells = gdf['Census_Wells'].sum()
    p_rural_national = rural_wells / total_wells if total_wells > 0 else 0
    
    print(f"   Total rural wells: {rural_wells:,.0f}")
    print(f"   Total wells: {total_wells:,.0f}")
    print(f"   National p_rural: {p_rural_national:.2%}")
    
    print("\n" + "-"*80)
    print("STEP 7: Calculating Scenario 1 (Uniform Domestic Demand)...")
    print("-"*80)
    
    gdf['q_dom_Ls'] = (C_PC_DOMESTIC * gdf['H_Block']) / SECONDS_PER_DAY
    
    gdf['Q_S1_Ls'] = gdf['Discrepancy'] * gdf['q_dom_Ls']
    gdf['Q_S1_m3_day'] = gdf['Q_S1_Ls'] * SECONDS_PER_DAY * LITERS_TO_M3
    gdf['Q_S1_m3_year'] = gdf['Q_S1_Ls'] * SECONDS_PER_YEAR * LITERS_TO_M3
    
    total_Q_S1_Ls = gdf['Q_S1_Ls'].sum()
    total_Q_S1_m3_year = gdf['Q_S1_m3_year'].sum()
    
    print(f"   Parameters: C_pc = {C_PC_DOMESTIC} L/person/day")
    print(f"   Total unregistered extraction (S1):")
    print(f"      - {total_Q_S1_Ls:,.2f} L/s")
    print(f"      - {total_Q_S1_m3_year:,.0f} m³/year")
    print(f"      - {total_Q_S1_m3_year/1e6:,.2f} million m³/year")
    
    print("\n" + "-"*80)
    print("STEP 8: Calculating Scenario 2 (Stratified Demand)...")
    print("-"*80)
    
    gdf['q_enhanced_Ls'] = (C_PC_ENHANCED * gdf['H_Block']) / SECONDS_PER_DAY
    
    gdf['q_S2_Ls'] = (gdf['p_exclusive'] * gdf['q_enhanced_Ls'] + 
                      (1 - gdf['p_exclusive']) * gdf['q_dom_Ls'])
    
    gdf['Q_S2_Ls'] = gdf['Discrepancy'] * gdf['q_S2_Ls']
    gdf['Q_S2_m3_day'] = gdf['Q_S2_Ls'] * SECONDS_PER_DAY * LITERS_TO_M3
    gdf['Q_S2_m3_year'] = gdf['Q_S2_Ls'] * SECONDS_PER_YEAR * LITERS_TO_M3
    
    total_Q_S2_Ls = gdf['Q_S2_Ls'].sum()
    total_Q_S2_m3_year = gdf['Q_S2_m3_year'].sum()
    
    print(f"   Parameters: C_enhanced = {C_PC_ENHANCED} L/person/day")
    print(f"   Total unregistered extraction (S2):")
    print(f"      - {total_Q_S2_Ls:,.2f} L/s")
    print(f"      - {total_Q_S2_m3_year:,.0f} m³/year")
    print(f"      - {total_Q_S2_m3_year/1e6:,.2f} million m³/year")
    print(f"   Increase over S1: {((total_Q_S2_m3_year/total_Q_S1_m3_year)-1)*100:+.1f}%")
    
    print("\n" + "-"*80)
    print("STEP 9: Calculating Scenario 3 (Mixed-Use with Productive)...")
    print("-"*80)
    
    gdf['Rural_Productive_Ls'] = P_PRODUCTIVE * Q_PRODUCTIVE_LS
    gdf['Rural_Domestic_Ls'] = (1 - P_PRODUCTIVE) * gdf['q_S2_Ls']
    gdf['Rural_Rate_Ls'] = gdf['Rural_Productive_Ls'] + gdf['Rural_Domestic_Ls']
    
    gdf['Urban_Rate_Ls'] = gdf['q_S2_Ls']
    
    gdf['q_S3_Ls'] = (gdf['p_rural'] * gdf['Rural_Rate_Ls'] + 
                      (1 - gdf['p_rural']) * gdf['Urban_Rate_Ls'])
    
    gdf['Q_S3_Ls'] = gdf['Discrepancy'] * gdf['q_S3_Ls']
    gdf['Q_S3_m3_day'] = gdf['Q_S3_Ls'] * SECONDS_PER_DAY * LITERS_TO_M3
    gdf['Q_S3_m3_year'] = gdf['Q_S3_Ls'] * SECONDS_PER_YEAR * LITERS_TO_M3
    
    total_Q_S3_Ls = gdf['Q_S3_Ls'].sum()
    total_Q_S3_m3_year = gdf['Q_S3_m3_year'].sum()
    
    print(f"   Parameters: p_productive = {P_PRODUCTIVE:.0%}, q_productive = {Q_PRODUCTIVE_LS} L/s")
    print(f"   Total unregistered extraction (S3):")
    print(f"      - {total_Q_S3_Ls:,.2f} L/s")
    print(f"      - {total_Q_S3_m3_year:,.0f} m³/year")
    print(f"      - {total_Q_S3_m3_year/1e6:,.2f} million m³/year")
    print(f"   Increase over S1: {((total_Q_S3_m3_year/total_Q_S1_m3_year)-1)*100:+.1f}%")
    print(f"   Increase over S2: {((total_Q_S3_m3_year/total_Q_S2_m3_year)-1)*100:+.1f}%")
    
    print("\n" + "-"*80)
    print("STEP 10: Calculating total extraction (DGA + Unregistered)...")
    print("-"*80)
    
    if 'DGA_Caudal_Ls' in gdf.columns:
        gdf['Q_DGA_Ls'] = gdf['DGA_Caudal_Ls']
        gdf['Q_DGA_m3_year'] = gdf['DGA_Caudal_Ls'] * SECONDS_PER_YEAR * LITERS_TO_M3
    else:
        gdf['Q_DGA_Ls'] = 0
        gdf['Q_DGA_m3_year'] = 0
    
    gdf['Q_Total_S1_Ls'] = gdf['Q_DGA_Ls'] + gdf['Q_S1_Ls']
    gdf['Q_Total_S2_Ls'] = gdf['Q_DGA_Ls'] + gdf['Q_S2_Ls']
    gdf['Q_Total_S3_Ls'] = gdf['Q_DGA_Ls'] + gdf['Q_S3_Ls']
    
    gdf['Q_Total_S1_m3y'] = gdf['Q_DGA_m3_year'] + gdf['Q_S1_m3_year']
    gdf['Q_Total_S2_m3y'] = gdf['Q_DGA_m3_year'] + gdf['Q_S2_m3_year']
    gdf['Q_Total_S3_m3y'] = gdf['Q_DGA_m3_year'] + gdf['Q_S3_m3_year']
    
    gdf['Pct_Unreg_S1'] = np.where(
        gdf['Q_Total_S1_m3y'] > 0,
        (gdf['Q_S1_m3_year'] / gdf['Q_Total_S1_m3y']) * 100,
        0
    )
    gdf['Pct_Unreg_S2'] = np.where(
        gdf['Q_Total_S2_m3y'] > 0,
        (gdf['Q_S2_m3_year'] / gdf['Q_Total_S2_m3y']) * 100,
        0
    )
    gdf['Pct_Unreg_S3'] = np.where(
        gdf['Q_Total_S3_m3y'] > 0,
        (gdf['Q_S3_m3_year'] / gdf['Q_Total_S3_m3y']) * 100,
        0
    )
    
    Q_DGA_national = gdf['Q_DGA_m3_year'].sum()
    Q_Total_S1 = gdf['Q_Total_S1_m3y'].sum()
    Q_Total_S2 = gdf['Q_Total_S2_m3y'].sum()
    Q_Total_S3 = gdf['Q_Total_S3_m3y'].sum()
    
    print(f"\n   === INTEGRATED NATIONAL EXTRACTION ===")
    print(f"   {'Source':<35} {'Flow (L/s)':>15} {'Volume (Mm³/yr)':>18}")
    print(f"   {'-'*70}")
    print(f"   {'DGA Registered':<35} {gdf['Q_DGA_Ls'].sum():>15,.2f} {Q_DGA_national/1e6:>18,.2f}")
    print(f"   {'Unregistered (Scenario 1)':<35} {total_Q_S1_Ls:>15,.2f} {total_Q_S1_m3_year/1e6:>18,.2f}")
    print(f"   {'Unregistered (Scenario 2)':<35} {total_Q_S2_Ls:>15,.2f} {total_Q_S2_m3_year/1e6:>18,.2f}")
    print(f"   {'Unregistered (Scenario 3)':<35} {total_Q_S3_Ls:>15,.2f} {total_Q_S3_m3_year/1e6:>18,.2f}")
    print(f"   {'-'*70}")
    print(f"   {'TOTAL (DGA + S1)':<35} {gdf['Q_Total_S1_Ls'].sum():>15,.2f} {Q_Total_S1/1e6:>18,.2f}")
    print(f"   {'TOTAL (DGA + S2)':<35} {gdf['Q_Total_S2_Ls'].sum():>15,.2f} {Q_Total_S2/1e6:>18,.2f}")
    print(f"   {'TOTAL (DGA + S3)':<35} {gdf['Q_Total_S3_Ls'].sum():>15,.2f} {Q_Total_S3/1e6:>18,.2f}")
    
    print("\n" + "-"*80)
    print("STEP 11: Preparing output columns...")
    print("-"*80)
    
    new_cols = [
        'Discrepancy', 'Ratio_Census_DGA',
        'H_Block', 'Used_Default_H',
        'p_exclusive', 'Pozos_Exclusivos', 'p_rural',
        'q_dom_Ls', 'Q_S1_Ls', 'Q_S1_m3_day', 'Q_S1_m3_year',
        'q_enhanced_Ls', 'q_S2_Ls', 'Q_S2_Ls', 'Q_S2_m3_day', 'Q_S2_m3_year',
        'Rural_Rate_Ls', 'Urban_Rate_Ls', 'q_S3_Ls', 'Q_S3_Ls', 'Q_S3_m3_day', 'Q_S3_m3_year',
        'Q_DGA_Ls', 'Q_DGA_m3_year',
        'Q_Total_S1_Ls', 'Q_Total_S2_Ls', 'Q_Total_S3_Ls',
        'Q_Total_S1_m3y', 'Q_Total_S2_m3y', 'Q_Total_S3_m3y',
        'Pct_Unreg_S1', 'Pct_Unreg_S2', 'Pct_Unreg_S3'
    ]
    
    print(f"   New columns added: {len(new_cols)}")
    print(f"   Total columns: {len(gdf.columns)}")
    
    print("\n" + "-"*80)
    print("STEP 12: Saving shapefile...")
    print("-"*80)
    
    new_col_shp_map = {
        'Discrepancy': 'Discrep',
        'Ratio_Census_DGA': 'Ratio_C_D',
        'H_Block': 'H_Block',
        'Used_Default_H': 'Used_Def_H',
        'p_exclusive': 'p_exclus',
        'Pozos_Exclusivos': 'Pozos_Exc',
        'p_rural': 'p_rural',
        'q_dom_Ls': 'q_dom_Ls',
        'Q_S1_Ls': 'Q_S1_Ls',
        'Q_S1_m3_day': 'Q_S1_m3d',
        'Q_S1_m3_year': 'Q_S1_m3y',
        'q_enhanced_Ls': 'q_enh_Ls',
        'q_S2_Ls': 'q_S2_Ls',
        'Q_S2_Ls': 'Q_S2_Ls',
        'Q_S2_m3_day': 'Q_S2_m3d',
        'Q_S2_m3_year': 'Q_S2_m3y',
        'Rural_Rate_Ls': 'Rur_Rate',
        'Urban_Rate_Ls': 'Urb_Rate',
        'q_S3_Ls': 'q_S3_Ls',
        'Q_S3_Ls': 'Q_S3_Ls',
        'Q_S3_m3_day': 'Q_S3_m3d',
        'Q_S3_m3_year': 'Q_S3_m3y',
        'Q_DGA_Ls': 'Q_DGA_Ls',
        'Q_DGA_m3_year': 'Q_DGA_m3y',
        'Q_Total_S1_Ls': 'QTot_S1Ls',
        'Q_Total_S2_Ls': 'QTot_S2Ls',
        'Q_Total_S3_Ls': 'QTot_S3Ls',
        'Q_Total_S1_m3y': 'QTot_S1m3',
        'Q_Total_S2_m3y': 'QTot_S2m3',
        'Q_Total_S3_m3y': 'QTot_S3m3',
        'Pct_Unreg_S1': 'Pct_Un_S1',
        'Pct_Unreg_S2': 'Pct_Un_S2',
        'Pct_Unreg_S3': 'Pct_Un_S3'
    }
    
    all_col_map = {**REVERSE_COL_MAP, **new_col_shp_map}
    
    gdf_shp = gdf.copy()
    
    rename_dict = {}
    for col in gdf_shp.columns:
        if col == 'geometry':
            continue
        if col in all_col_map:
            rename_dict[col] = all_col_map[col]
        elif len(col) > 10:
            rename_dict[col] = col[:10]
    
    gdf_shp = gdf_shp.rename(columns=rename_dict)
    
    shp_path = os.path.join(OUTPUT_FOLDER, 'Shapefiles', 'Census_DGA_WaterConsumption_Estimates.shp')
    gdf_shp.to_file(shp_path, driver='ESRI Shapefile', encoding='utf-8')
    print(f"   ✅ Shapefile saved: {shp_path}")
    
    print("\n" + "-"*80)
    print("STEP 13: Saving CSV...")
    print("-"*80)
    
    df_csv = gdf.drop(columns='geometry').copy()
    
    float_cols = df_csv.select_dtypes(include=[np.floating]).columns
    for col in float_cols:
        df_csv[col] = df_csv[col].round(4)
    
    csv_path = os.path.join(OUTPUT_FOLDER, 'CSV', 'Census_DGA_WaterConsumption_Estimates.csv')
    df_csv.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"   ✅ CSV saved: {csv_path}")
    
    print("\n" + "-"*80)
    print("STEP 14: Saving aggregated Excel files...")
    print("-"*80)
    
    def aggregate_results(df, group_col, name_col):
        agg_dict = {
            'Census_Wells': 'sum',
            'DGA_WaterRights': 'sum',
            'Discrepancy': 'sum',
            'Personas_24': 'sum',
            'Viviendas_24': 'sum',
            'Is_Rural': 'sum',
            'Q_S1_Ls': 'sum',
            'Q_S1_m3_year': 'sum',
            'Q_S2_Ls': 'sum',
            'Q_S2_m3_year': 'sum',
            'Q_S3_Ls': 'sum',
            'Q_S3_m3_year': 'sum',
            'Q_DGA_Ls': 'sum',
            'Q_DGA_m3_year': 'sum',
            'Q_Total_S1_Ls': 'sum',
            'Q_Total_S1_m3y': 'sum',
            'Q_Total_S2_Ls': 'sum',
            'Q_Total_S2_m3y': 'sum',
            'Q_Total_S3_Ls': 'sum',
            'Q_Total_S3_m3y': 'sum'
        }
        
        agg_dict = {k: v for k, v in agg_dict.items() if k in df.columns}
        
        agg = df.groupby(group_col, dropna=False).agg(agg_dict).reset_index()
        agg = agg.rename(columns={group_col: name_col})
        
        block_counts = df.groupby(group_col, dropna=False).size().reset_index(name='N_Blocks')
        block_counts = block_counts.rename(columns={group_col: name_col})
        agg = agg.merge(block_counts, on=name_col, how='left')
        
        if 'Q_Total_S3_m3y' in agg.columns and 'Q_S3_m3_year' in agg.columns:
            agg['Pct_Unregistered_S3'] = np.where(
                agg['Q_Total_S3_m3y'] > 0,
                (agg['Q_S3_m3_year'] / agg['Q_Total_S3_m3y']) * 100,
                0
            )
        
        return agg
    
    aggregation_levels = [
        ('Region_Name', 'Region'),
        ('Muni_Name', 'Municipality'),
        ('Cuenca_Name', 'Basin'),
        ('SHAC_Name', 'SHAC')
    ]
    
    aggregated_results = {}
    for group_col, name in aggregation_levels:
        if group_col in gdf.columns:
            agg_df = aggregate_results(gdf, group_col, name)
            agg_df = agg_df.sort_values('Q_S3_m3_year', ascending=False)
            aggregated_results[name] = agg_df
            print(f"   Aggregated at {name} level: {len(agg_df)} units")
    
    national_summary = pd.DataFrame({
        'Parameter': [
            'Total Census Blocks',
            'Total Census Wells',
            'Total DGA Rights',
            'Total Discrepancy (Unregistered)',
            'Ratio Census/DGA',
            'National avg H (persons/household)',
            'p_exclusive (national)',
            'p_rural (national)',
            '',
            '--- SCENARIO 1: UNIFORM DOMESTIC ---',
            'Unregistered Q (S1) [L/s]',
            'Unregistered Q (S1) [Mm³/year]',
            '',
            '--- SCENARIO 2: STRATIFIED ---',
            'Unregistered Q (S2) [L/s]',
            'Unregistered Q (S2) [Mm³/year]',
            '',
            '--- SCENARIO 3: MIXED-USE ---',
            'Unregistered Q (S3) [L/s]',
            'Unregistered Q (S3) [Mm³/year]',
            '',
            '--- TOTAL EXTRACTION ---',
            'DGA Registered [L/s]',
            'DGA Registered [Mm³/year]',
            'Total (DGA+S1) [Mm³/year]',
            'Total (DGA+S2) [Mm³/year]',
            'Total (DGA+S3) [Mm³/year]',
            '% Unregistered (S1)',
            '% Unregistered (S2)',
            '% Unregistered (S3)'
        ],
        'Value': [
            len(gdf),
            total_census,
            total_dga,
            total_discrepancy,
            total_census / total_dga if total_dga > 0 else 0,
            H_AVG_NATIONAL,
            p_exclusive_national,
            p_rural_national,
            '',
            '',
            total_Q_S1_Ls,
            total_Q_S1_m3_year / 1e6,
            '',
            '',
            total_Q_S2_Ls,
            total_Q_S2_m3_year / 1e6,
            '',
            '',
            total_Q_S3_Ls,
            total_Q_S3_m3_year / 1e6,
            '',
            '',
            gdf['Q_DGA_Ls'].sum(),
            Q_DGA_national / 1e6,
            Q_Total_S1 / 1e6,
            Q_Total_S2 / 1e6,
            Q_Total_S3 / 1e6,
            (total_Q_S1_m3_year / Q_Total_S1) * 100 if Q_Total_S1 > 0 else 0,
            (total_Q_S2_m3_year / Q_Total_S2) * 100 if Q_Total_S2 > 0 else 0,
            (total_Q_S3_m3_year / Q_Total_S3) * 100 if Q_Total_S3 > 0 else 0
        ]
    })
    
    params_df = pd.DataFrame({
        'Parameter': [
            'C_PC_DOMESTIC', 'C_PC_ENHANCED', 'H_AVG_DEFAULT', 'H_AVG_NATIONAL',
            'P_PRODUCTIVE', 'Q_PRODUCTIVE_LS', 'p_exclusive_national', 'p_rural_national'
        ],
        'Value': [
            C_PC_DOMESTIC, C_PC_ENHANCED, H_AVG_DEFAULT, H_AVG_NATIONAL,
            P_PRODUCTIVE, Q_PRODUCTIVE_LS, p_exclusive_national, p_rural_national
        ],
        'Unit': [
            'L/person/day', 'L/person/day', 'persons/household', 'persons/household',
            'fraction', 'L/s', 'fraction', 'fraction'
        ],
        'Description': [
            'Standard domestic consumption',
            'Enhanced consumption (exclusive wells)',
            'Default household size',
            'National average household size',
            'Fraction rural wells with productive use',
            'Productive well extraction rate',
            'Proportion exclusive well dependence',
            'Proportion wells in rural areas'
        ]
    })
    
    col_definitions = pd.DataFrame({
        'Column': [
            '--- DISCREPANCY ---',
            'Discrepancy', 'Ratio_Census_DGA',
            '',
            '--- HOUSEHOLD SIZE ---',
            'H_Block', 'Used_Default_H',
            '',
            '--- PROPORTIONS ---',
            'p_exclusive', 'p_rural',
            '',
            '--- SCENARIO 1 (L/s) ---',
            'q_dom_Ls', 'Q_S1_Ls',
            '',
            '--- SCENARIO 2 (L/s) ---',
            'q_S2_Ls', 'Q_S2_Ls',
            '',
            '--- SCENARIO 3 (L/s) ---',
            'q_S3_Ls', 'Q_S3_Ls',
            '',
            '--- ANNUAL VOLUME (m³/year) ---',
            'Q_S1_m3_year', 'Q_S2_m3_year', 'Q_S3_m3_year',
            '',
            '--- TOTAL EXTRACTION ---',
            'Q_Total_S1_m3y', 'Q_Total_S2_m3y', 'Q_Total_S3_m3y',
            'Pct_Unreg_S1', 'Pct_Unreg_S2', 'Pct_Unreg_S3'
        ],
        'Description': [
            '',
            'Census Wells - DGA Rights (clipped to 0)',
            'Census Wells / DGA Rights',
            '',
            '',
            'Persons per household (block level)',
            'Flag: 1 if using default H value',
            '',
            '',
            'Proportion exclusive well dependence',
            'Proportion in rural areas (Is_Rural)',
            '',
            '',
            'Domestic rate per well [L/s]',
            'Total unregistered extraction S1 [L/s]',
            '',
            '',
            'Stratified rate per well [L/s]',
            'Total unregistered extraction S2 [L/s]',
            '',
            '',
            'Mixed-use rate per well [L/s]',
            'Total unregistered extraction S3 [L/s]',
            '',
            '',
            'Annual S1 volume', 'Annual S2 volume', 'Annual S3 volume',
            '',
            '',
            'DGA + S1 annual volume', 'DGA + S2 annual volume', 'DGA + S3 annual volume',
            'Percent unregistered S1', 'Percent unregistered S2', 'Percent unregistered S3'
        ]
    })
    
    excel_path = os.path.join(OUTPUT_FOLDER, 'Excel', 'Water_Consumption_Estimates_All_Levels.xlsx')
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        national_summary.to_excel(writer, sheet_name='National_Summary', index=False)
        params_df.to_excel(writer, sheet_name='Parameters', index=False)
        col_definitions.to_excel(writer, sheet_name='Column_Definitions', index=False)
        
        df_csv.head(50000).to_excel(writer, sheet_name='Block_Level_Sample', index=False)
        
        for name, agg_df in aggregated_results.items():
            sheet_name = f'{name}_Level'[:31]
            agg_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"   ✅ Excel saved: {excel_path}")
    
    print("\n" + "-"*80)
    print("STEP 15: Generating report...")
    print("-"*80)
    
    report_lines = []
    report_lines.append("="*100)
    report_lines.append("GROUNDWATER EXTRACTION ESTIMATION REPORT")
    report_lines.append("From Census 2024 vs DGA Water Rights Comparison")
    report_lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report_lines.append("="*100)
    
    report_lines.append("\nINPUT DATA:")
    report_lines.append(f"  Source shapefile: {INPUT_SHAPEFILE}")
    report_lines.append(f"  Census blocks analyzed: {len(gdf):,}")
    
    report_lines.append("\nPARAMETERS USED:")
    report_lines.append(f"  C_PC_DOMESTIC: {C_PC_DOMESTIC} L/person/day")
    report_lines.append(f"  C_PC_ENHANCED: {C_PC_ENHANCED} L/person/day")
    report_lines.append(f"  H_AVG_DEFAULT: {H_AVG_DEFAULT} persons/household")
    report_lines.append(f"  H_AVG_NATIONAL: {H_AVG_NATIONAL:.2f} persons/household")
    report_lines.append(f"  P_PRODUCTIVE: {P_PRODUCTIVE:.0%}")
    report_lines.append(f"  Q_PRODUCTIVE_LS: {Q_PRODUCTIVE_LS} L/s")
    report_lines.append(f"  p_exclusive (national): {p_exclusive_national:.2%}")
    report_lines.append(f"  p_rural (national): {p_rural_national:.2%}")
    
    report_lines.append("\n" + "="*80)
    report_lines.append("NATIONAL SUMMARY")
    report_lines.append("="*80)
    report_lines.append(f"Total Census Wells: {total_census:,.0f}")
    report_lines.append(f"Total DGA Rights: {total_dga:,.0f}")
    report_lines.append(f"Discrepancy (Unregistered): {total_discrepancy:,.0f}")
    report_lines.append(f"Ratio Census/DGA: {total_census/total_dga:.2f}x" if total_dga > 0 else "Ratio: N/A")
    
    report_lines.append("\n--- EXTRACTION ESTIMATES ---")
    report_lines.append(f"{'Scenario':<25} {'Flow (L/s)':>15} {'Volume (Mm³/yr)':>18}")
    report_lines.append("-"*60)
    report_lines.append(f"{'DGA Registered':<25} {gdf['Q_DGA_Ls'].sum():>15,.2f} {Q_DGA_national/1e6:>18,.2f}")
    report_lines.append(f"{'S1: Uniform Domestic':<25} {total_Q_S1_Ls:>15,.2f} {total_Q_S1_m3_year/1e6:>18,.2f}")
    report_lines.append(f"{'S2: Stratified':<25} {total_Q_S2_Ls:>15,.2f} {total_Q_S2_m3_year/1e6:>18,.2f}")
    report_lines.append(f"{'S3: Mixed-Use':<25} {total_Q_S3_Ls:>15,.2f} {total_Q_S3_m3_year/1e6:>18,.2f}")
    report_lines.append("-"*60)
    report_lines.append(f"{'TOTAL (DGA + S3)':<25} {gdf['Q_Total_S3_Ls'].sum():>15,.2f} {Q_Total_S3/1e6:>18,.2f}")
    
    report_lines.append("\n--- KEY FINDINGS ---")
    report_lines.append(f"• Unregistered wells add {(total_Q_S1_m3_year/Q_DGA_national)*100:.0f}% - {(total_Q_S3_m3_year/Q_DGA_national)*100:.0f}% to registered extraction" if Q_DGA_national > 0 else "")
    report_lines.append(f"• {p_exclusive_national*100:.1f}% of well users have NO public network access")
    report_lines.append(f"• {p_rural_national*100:.1f}% of all wells are in RURAL areas")
    
    report_lines.append("\n" + "="*80)
    report_lines.append("OUTPUT FILES")
    report_lines.append("="*80)
    report_lines.append(f"Shapefile: {shp_path}")
    report_lines.append(f"CSV: {csv_path}")
    report_lines.append(f"Excel: {excel_path}")
    
    report_lines.append("\n" + "="*80)
    report_lines.append("SHAPEFILE COLUMNS (NEW)")
    report_lines.append("="*80)
    report_lines.append("Discrep: Discrepancy (Census - DGA, clipped to 0)")
    report_lines.append("H_Block: Household size per block")
    report_lines.append("p_exclus: Exclusive well dependence proportion")
    report_lines.append("p_rural: Rural proportion (Is_Rural)")
    report_lines.append("Q_S1_Ls, Q_S1_m3y: Scenario 1 extraction (L/s, m³/year)")
    report_lines.append("Q_S2_Ls, Q_S2_m3y: Scenario 2 extraction (L/s, m³/year)")
    report_lines.append("Q_S3_Ls, Q_S3_m3y: Scenario 3 extraction (L/s, m³/year)")
    report_lines.append("QTot_S*: Total extraction (DGA + Scenario)")
    report_lines.append("Pct_Un_S*: Percentage unregistered")
    
    report_lines.append("\n" + "="*100)
    report_lines.append("END OF REPORT")
    report_lines.append("="*100)
    
    report_path = os.path.join(OUTPUT_FOLDER, 'Reports', 
                              f'Water_Consumption_Report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt')
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_lines))
    print(f"   ✅ Report saved: {report_path}")
    
    print("\n" + "="*100)
    print("ANALYSIS COMPLETE!")
    print("="*100)
    print(f"\nOutput folder: {OUTPUT_FOLDER}")
    print(f"\nFiles generated:")
    print(f"  - Shapefile: Shapefiles/Census_DGA_WaterConsumption_Estimates.shp")
    print(f"  - CSV: CSV/Census_DGA_WaterConsumption_Estimates.csv")
    print(f"  - Excel: Excel/Water_Consumption_Estimates_All_Levels.xlsx")
    print(f"  - Report: Reports/Water_Consumption_Report_*.txt")
    
    print(f"\n--- KEY RESULTS ---")
    print(f"Unregistered wells: {total_discrepancy:,.0f}")
    print(f"Unregistered extraction (S1-S3): {total_Q_S1_m3_year/1e6:.1f} - {total_Q_S3_m3_year/1e6:.1f} Mm³/year")
    print(f"Total extraction (DGA + S3): {Q_Total_S3/1e6:.1f} Mm³/year")
    
    print(f"\nFinished: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if __name__ == "__main__":
    main()